In [13]:
# Cell 1: Install Required Packages for Google Colab
!pip install -q sentence-transformers
!pip install -q faiss-cpu
!pip install -q langchain
!pip install -q chromadb

In [14]:
# Essential imports
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline
import faiss
import warnings
from typing import List, Dict
warnings.filterwarnings('ignore')

print("Setup complete!")

Setup complete!


In [22]:
# Import library yang dibutuhkan
import pandas as pd

# Membaca dataset fashion dari file CSV
fashion_data = pd.read_csv("/content/fashion_dataset.csv")

# Membuat dataset styling tips
styling_tips = pd.DataFrame({
    'style_type': [
        'Casual',
        'Business',
        'Evening',
        'Smart Casual',
        'Formal'
    ],
    'tips': [
        'For casual outfits, focus on comfort while maintaining style. Pair basic pieces like t-shirts with well-fitted jeans.',
        'Business attire should be professional and polished. A blazer can instantly elevate any office outfit.',
        'Evening wear calls for more dramatic pieces. Consider leather pants or sequined items for a night out.',
        'Smart casual combines comfort with sophistication. Try pairing a cashmere sweater with tailored pants.',
        'Formal occasions require elegant pieces. Choose floor-length gowns or sophisticated cocktail dresses.'
    ]
})

print("Datasets created successfully!")


Datasets created successfully!


In [23]:
fashion_data.head()

,id,name,brand,category,description,style,price
0,1,Classic Shorts,Nike,Shorts,Breathable shorts perfect for warm weather and...,Formal,60.40
1,2,Slim Fit Jacket,Balenciaga,Jacket,Warm and cozy jacket for colder seasons.,Formal,262.11
2,3,Oversized Blazer,Balenciaga,Blazer,Stylish blazer ideal for professional or busin...,Formal,105.04
3,4,Elegant Shorts,Gucci,Shorts,Breathable shorts perfect for warm weather and...,Formal,494.44
4,5,Modern Tracksuit,Balenciaga,Tracksuit,Sporty tracksuit for exercise or casual lounging.,Casual,104.01


In [24]:
styling_tips.head()

,style_type,tips
0,Casual,"For casual outfits, focus on comfort while mai..."
1,Business,Business attire should be professional and pol...
2,Evening,Evening wear calls for more dramatic pieces. C...
3,Smart Casual,Smart casual combines comfort with sophisticat...
4,Formal,Formal occasions require elegant pieces. Choos...


In [25]:
# Cell 4: Create Fashion RAG System
class FashionRAG:
    def __init__(self):
        # Embedding Model: Uses 'all-MiniLM-L6-v2' because it's efficient and good at understanding semantic meaning in text
        # FAISS Index: Uses dimension 384 because that's what the MiniLM model outputs. IndexFlatL2 is chosen for L2 (Euclidean) distance calculation
        #Storage Lists: Keeps original data and text representations separate for easy retrieval
        # Initialize the embedding model
        self.model = SentenceTransformer('all-MiniLM-L6-v2')

        # Initialize FAISS index
        self.dimension = 384  # embedding dimension for 'all-MiniLM-L6-v2'
        self.index = faiss.IndexFlatL2(self.dimension)

        # Storage for data
        self.fashion_items = []
        self.item_texts = []

    def add_items(self, fashion_df: pd.DataFrame, tips_df: pd.DataFrame):
        """Add fashion items and styling tips to the system"""
        # This is the RAG part where we:
        # Format Data: Combines product info and styling tips into structured text
        # Create Embeddings: Converts text to numerical vectors (word embeddings)
        # Build Index: Stores embeddings in FAISS for fast similarity search
        # RAG Integration: Merges product catalog with styling knowledge
        # Create text representations
        for _, row in fashion_df.iterrows():
            text = f"Item: {row['name']}. Description: {row['description']}. " \
                   f"Style: {row['style']}. Price: ${row['price']}"
            self.item_texts.append(text)
            self.fashion_items.append(row.to_dict())

        # Add styling tips
        for _, row in tips_df.iterrows():
            text = f"Style Guide - {row['style_type']}: {row['tips']}"
            self.item_texts.append(text)
            self.fashion_items.append({
                'name': f"Style Guide - {row['style_type']}",
                'description': row['tips'],
                'style': row['style_type'],
                'price': None
            })

        # Create embeddings and add to index
        embeddings = self.model.encode(self.item_texts)
        self.index.add(embeddings.astype('float32'))

        print(f"Added {len(self.item_texts)} items to the system")

    def search(self, query: str, k: int = 5) -> List[Dict]:
        """Search for relevant items based on query"""
        #This is the Retrieval part where:
        #Query Processing: Converts user query to word embedding
        #Similarity Search: Uses FAISS to find closest matches between the query and the available data
        #Score Calculation: Converts distances to similarity scores
        #Result Assembly: Combines items with their relevance scores
        # Create query embedding
        query_vector = self.model.encode([query])

        # Search in FAISS index
        distances, indices = self.index.search(query_vector.astype('float32'), k)

        # Get results
        results = []
        for idx, distance in zip(indices[0], distances[0]):
            item = self.fashion_items[idx]
            results.append({
                'item': item,
                'relevance_score': 1 / (1 + distance),  # Convert distance to similarity score
                'text': self.item_texts[idx]
            })

        return results

    def get_recommendations(self, query: str) -> str:
        # This is the Generation part where:
        # Result Processing: Distinguishes between products and styling tips
        # Format Differentiation: Handles products and tips differently
        # Score Display: Shows how relevant each item is to the query
        """Get formatted recommendations based on query"""
        results = self.search(query)

        # Format output
        output = f"Recommendations for: {query}\n"
        output += "=" * 50 + "\n\n"

        for i, result in enumerate(results, 1):
            item = result['item']
            output += f"{i}. "

            if item['price'] is not None:  # Fashion item
                output += f"{item['name']}\n"
                output += f"   Style: {item['style']}\n"
                output += f"   Price: ${item['price']}\n"
                output += f"   Description: {item['description']}\n"
            else:  # Styling tip
                output += f"{item['description']}\n"

            output += f"   Relevance Score: {result['relevance_score']:.2f}\n\n"

        return output

In [28]:
# Cell 5: Initialize and Test System
# Create RAG system
rag = FashionRAG()
rag.add_items(fashion_data, styling_tips)

# Test queries
test_queries = [
    "What can I wear to a creative industry job interview?",
    "Recommend a laid-back look for a beachside brunch.",
    "Show me party outfits that look expensive but are affordable.",
    "What’s appropriate attire for an upscale art gallery opening?",
    "Suggest a cozy yet trendy outfit for a mountain getaway."
]

# Run tests
for query in test_queries:
    print(f"\nQuery: {query}")
    print("-" * 60)
    print(rag.get_recommendations(query))
    print("=" * 60)

Added 105 items to the system

Query: What can I wear to a creative industry job interview?
------------------------------------------------------------
Recommendations for: What can I wear to a creative industry job interview?

1. Business attire should be professional and polished. A blazer can instantly elevate any office outfit.
   Relevance Score: 0.51

2. Smart casual combines comfort with sophistication. Try pairing a cashmere sweater with tailored pants.
   Relevance Score: 0.49

3. Formal occasions require elegant pieces. Choose floor-length gowns or sophisticated cocktail dresses.
   Relevance Score: 0.47

4. For casual outfits, focus on comfort while maintaining style. Pair basic pieces like t-shirts with well-fitted jeans.
   Relevance Score: 0.46

5. Evening wear calls for more dramatic pieces. Consider leather pants or sequined items for a night out.
   Relevance Score: 0.45



Query: Recommend a laid-back look for a beachside brunch.
-------------------------------------